# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID                    City      Lat       Lng  Max Temp  Humidity  \
0        0             nar'yan-mar  67.6713   53.0870     -0.50        95   
1        1                akureyri  65.6835  -18.0878     -0.04       100   
2        2                 iqaluit  63.7506  -68.5145     -3.15        63   
3        3  happy valley-goose bay  53.3168  -60.3315     -3.83        54   
4        4           atafu village  -8.5421 -172.5159     29.01        71   

   Cloudiness  Wind Speed Country        Date  
0         100        9.39      RU  1699244915  
1         100        4.63      IS  1699244915  
2         100       12.86      CA  1699244915  
3          20        5.66      CA  1699244916  
4         100        1.31      TK  1699244917

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
%%capture --no-display
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale=1,
    alpha = 0.5,
    color = 'City',
    frame_width = 600,
    frame_height = 500,
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [30]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21), :]
narrow_df = narrow_df.loc[narrow_df['Wind Speed'] < 4.5,:]
narrow_df = narrow_df.loc[narrow_df['Cloudiness'] == 0, :]

# Drop any rows with null values
narrow_city_df = narrow_df.dropna()

# Display sample data
print(f'In total, {len(narrow_city_df)} cities were found for your ideal weather condition')
narrow_city_df

In total, 16 cities were found for your ideal weather condition


City_ID             City      Lat      Lng  Max Temp  Humidity  \
50        50  remire-montjoly   4.9167 -52.2667     26.02        94   
73        73        taoudenni  22.6783  -3.9836     26.41        16   
208      208          matoury   4.8483 -52.3317     25.98        94   
287      287       siwa oasis  29.2041  25.5195     22.54        29   
294      294          reggane  26.7158   0.1714     24.06        15   
303      303             tura  25.5198  90.2201     26.60        52   
334      334           waddan  29.1614  16.1390     21.36        22   
407      407        i-n-salah  27.1935   2.4607     23.81        14   
409      409          salalah  17.0151  54.0924     26.05        78   
419      419     aransas pass  27.9095 -97.1500     23.88        86   
429      429             ghat  24.9647  10.1728     21.05        16   
438      438          saidpur  25.7777  88.8917     26.91        56   
440      440           tahoua  14.8888   5.2692     23.99        22   
475      475            aswan  24.0934  32.9070     23.61        35   
530      530       rio branco  -9.9747 -67.8100     25.34        94   
536      536      porto velho  -8.7619 -63.9039     26.03        89   

     Cloudiness  Wind Speed Country        Date  
50            0        1.54      GF  1699244939  
73            0        3.26      ML  1699244947  
208           0        1.54      GF  1699244989  
287           0        1.51      EG  1699245014  
294           0        1.90      DZ  1699245016  
303           0        0.30      IN  1699245019  
334           0        1.62      LY  1699245032  
407           0        1.87      DZ  1699245055  
409           0        1.03      OM  1699245056  
419           0        3.60      US  1699245058  
429           0        1.90      LY  1699245061  
438           0        2.54      BD  1699245064  
440           0        3.09      NE  1699245065  
475           0        3.09      EG  1699245077  
530           0        1.54      BR  1699245067  
536           0        0.00      BR  1699245081

### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df


City Country      Lat      Lng  Humidity Hotel Name
50   remire-montjoly      GF   4.9167 -52.2667        94           
73         taoudenni      ML  22.6783  -3.9836        16           
208          matoury      GF   4.8483 -52.3317        94           
287       siwa oasis      EG  29.2041  25.5195        29           
294          reggane      DZ  26.7158   0.1714        15           
303             tura      IN  25.5198  90.2201        52           
334           waddan      LY  29.1614  16.1390        22           
407        i-n-salah      DZ  27.1935   2.4607        14           
409          salalah      OM  17.0151  54.0924        78           
419     aransas pass      US  27.9095 -97.1500        86           
429             ghat      LY  24.9647  10.1728        16           
438          saidpur      BD  25.7777  88.8917        56           
440           tahoua      NE  14.8888   5.2692        22           
475            aswan      EG  24.0934  32.9070        35           
530       rio branco      BR  -9.9747 -67.8100        94           
536      porto velho      BR  -8.7619 -63.9039        89

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [32]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
remire-montjoly - nearest hotel: Complexe Belova
taoudenni - nearest hotel: No hotel found
matoury - nearest hotel: La Marmotte
siwa oasis - nearest hotel: فندق الكيلانى
reggane - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
waddan - nearest hotel: No hotel found
i-n-salah - nearest hotel: باجودة
salalah - nearest hotel: Muscat International Hotel
aransas pass - nearest hotel: Executive Inn & Suites
ghat - nearest hotel: فندق تاسيلي
saidpur - nearest hotel: Hotel Amzadia International হোটেল আমজাদিয়া ইন্টারন্যাশনাল
tahoua - nearest hotel: Hotel Bungalow
aswan - nearest hotel: Yaseen hotel
rio branco - nearest hotel: No hotel found
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho


City Country      Lat      Lng  Humidity  \
50   remire-montjoly      GF   4.9167 -52.2667        94   
73         taoudenni      ML  22.6783  -3.9836        16   
208          matoury      GF   4.8483 -52.3317        94   
287       siwa oasis      EG  29.2041  25.5195        29   
294          reggane      DZ  26.7158   0.1714        15   
303             tura      IN  25.5198  90.2201        52   
334           waddan      LY  29.1614  16.1390        22   
407        i-n-salah      DZ  27.1935   2.4607        14   
409          salalah      OM  17.0151  54.0924        78   
419     aransas pass      US  27.9095 -97.1500        86   
429             ghat      LY  24.9647  10.1728        16   
438          saidpur      BD  25.7777  88.8917        56   
440           tahoua      NE  14.8888   5.2692        22   
475            aswan      EG  24.0934  32.9070        35   
530       rio branco      BR  -9.9747 -67.8100        94   
536      porto velho      BR  -8.7619 -63.9039        89   

                                            Hotel Name  
50                                     Complexe Belova  
73                                      No hotel found  
208                                        La Marmotte  
287                                      فندق الكيلانى  
294                                     No hotel found  
303                                     No hotel found  
334                                     No hotel found  
407                                             باجودة  
409                         Muscat International Hotel  
419                             Executive Inn & Suites  
429                                        فندق تاسيلي  
438  Hotel Amzadia International হোটেল আমজাদিয়া ইন্...  
440                                     Hotel Bungalow  
475                                       Yaseen hotel  
530                                     No hotel found  
536               Hotel Slaviero Essential Porto Velho

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale=1,
    alpha = 0.5,
    color = 'City',
    frame_width = 600,
    frame_height = 500,
    hover_cols = ['Hotel Name', 'Country']
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

In [ ]:
vv